In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob

Using TensorFlow backend.


In [ ]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pandas as pd
import shutil,os
downloaded = drive.CreateFile({'id':"1B8Aodo2QlZLtwnNp1GxcVuC2bg2Egjsn"})
downloaded.GetContentFile('Train.csv')
train= pd.read_csv('Train.csv') 
# SORTING BY CATEGORY
train.sort_values(by=["category"],inplace=True)
# INDEX RESET
train.reset_index(drop=True, inplace=True)
# DATAFRAME CONTAINING CATEGORY AND NAMES
categ=train['category']
name=train["name"]
ori_parent="/content/drive/My Drive/Colab Notebooks/lego/train"
parent="/content/drive/My Drive/Colab Notebooks/lego/"
# BELOW CODE USED TO MAKE 16 DIFFERENT FOLDERS AND PLACE IMAGES ACCORDINGLY
# for i in range(4466):
  
#   ori_path=ori_parent+"/"+str(name[i])
#   path=parent+str(categ[i])+"/"
#   shutil.copy(ori_path,path)

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
train_path="/content/drive/My Drive/Colab Notebooks/lego"
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# DON'T TRAIN THE LAYERS SO AS TO PRESERVE PRETRAINED VALUES
for layer in vgg.layers:
  layer.trainable = False
x = Flatten()(vgg.output)
# CUSTOMISED PREDICTION LAYER
prediction = Dense(16, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [13]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode="nearest")


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


r = model.fit_generator(
  training_set,
  
  epochs=10,
  steps_per_epoch=140,
  )



Found 4465 images belonging to 16 classes.
Epoch 1/10
102/140 [====================>.........] - ETA: 5:36 - loss: 0.8845 - accuracy: 0.6936

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 70 could not be retrieved. It could be because a worker has died.
  UserWarning)


140/140 [==============================] - 1302s 9s/step - loss: 0.7806 - accuracy: 0.7265
Epoch 2/10
140/140 [==============================] - 75s 533ms/step - loss: 0.4237 - accuracy: 0.8533
Epoch 3/10
140/140 [==============================] - 74s 530ms/step - loss: 0.2734 - accuracy: 0.9032
Epoch 4/10
140/140 [==============================] - 73s 522ms/step - loss: 0.2572 - accuracy: 0.9075
Epoch 5/10
140/140 [==============================] - 74s 527ms/step - loss: 0.2652 - accuracy: 0.9095
Epoch 6/10
140/140 [==============================] - 74s 525ms/step - loss: 0.1831 - accuracy: 0.9364
Epoch 7/10
140/140 [==============================] - 74s 526ms/step - loss: 0.1840 - accuracy: 0.9380
Epoch 8/10
140/140 [==============================] - 74s 531ms/step - loss: 0.1504 - accuracy: 0.9505
Epoch 9/10
140/140 [==============================] - 75s 535ms/step - loss: 0.1331 - accuracy: 0.9574
Epoch 10/10
140/140 [==============================] - 73s 523ms/step - loss: 0.1310 

In [ ]:
# os.mkdir("/content/drive/My Drive/Colab Notebooks/lego/pred_data")
# files=os.listdir("/content/drive/My Drive/question_1_dataset/test/test")
# for file in files:
#    path="/content/drive/My Drive/question_1_dataset/test/test/"+str(file)
#    new_path="/content/drive/My Drive/Colab Notebooks/lego/pred_data/"
#    shutil.copy(path,new_path)

In [3]:
# NOW TO PREDICT LABELS FROM THE GIVEN TEST DATA SET
import numpy as np
from google.colab import files
from keras.preprocessing import image

files=os.listdir("/content/drive/My Drive/Colab Notebooks/pred_data")
i=0
for file in files:
 
  # predicting images
  print(file)
  path = "/content/drive/My Drive/Colab Notebooks/pred_data/"+str(file)
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images)
  cl=np.argmax(classes)
  print(cl+1)
  i=i+1
  

NameError: ignored